In [1]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import pickle
from geopy.geocoders import Nominatim


# librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# soporte para limpieza
import sys
sys.path.append("../")

import src.soporte as sp

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (15,10)

# Info Dstos
---
**mountain**
---

**name_id** -> el id de la ruta


**grade_mean** -> media de todas las ascensiones

Cambié un poco la calificación de la ascensión: si alguien decía que una ruta era 7a dura, entonces ponía un 7a/+, lo mismo con la calificación suave. Después de eso, calculé la mediana para todas las calificaciones de cada ruta (más robusta con outlayers)


**rating_total** -> Hice este cálculo basado en 3 características y tomando el primer componente del PCA:

comentario sentimiento
clasificación
recomendaciones


|**cluster** | Agrupé las rutas en 9 clusters diferentes que pueden ser más o menos identificados como|
|---|---|
|0 | Rutas blandas|
|1 | Rutas por alguna razón preferidas por mujeres|
|2 | Rutas famosas|
|3 | Rutas muy duras|
|4 | Rutas muy repetidas|
|5 | Chipped rutas, con tarifa blanda|
|6 | Rutas tradicionales, no chippeadas|
|7 | Rutas fáciles a vista, poco repetidas|
|8 | Rutas muy famosas pero no tan repetidas y no tan tradicionales|


---
**scaler**
---
date_first -> fecha de la primera ascensión

date_last -> fecha de la última ascensión

grades_first -> grado de la primera ascensión

grades_last -> grado de la última ascensión

years_cl -> años escalando

grades_count -> número de vías realizadas por escalador

year_first -> año de la primera ascensión

year_last -> año de la última ascensión

In [2]:
dif = pd.read_csv("../data/grades_conversion_table.csv", index_col=0)
dif

,grade_id,grade_fra
0,0,-
1,1,-
2,2,-
3,3,1
4,4,1a
...,...,...
80,80,9b+/9c
81,81,9c
82,82,9c/+
83,83,9c+


In [54]:
dif.iloc[28:30]

,grade_id,grade_fra
28,28,4+
29,29,5


In [3]:
mountain = pd.read_csv("../data/routes_rated.csv", index_col=0)

In [4]:
mountain.head(10)

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.250000,3,-0.045211
1,1,and,montserrat,prohibitivo,mehir,-1,49.000000,2,0.116464
2,2,and,montserrat,prohibitivo,pas de la discordia,0,49.000000,2,0.178722
3,3,and,tartareu,bombo suis,tenedor libre,0,44.333333,3,0.158449
4,4,arg,bandurrias,rincon,tendinitis,1,48.500000,0,0.075797
5,5,arg,bariloche,pared blanca,barbaroja,0,49.000000,1,-0.164902
6,6,arg,bariloche,pared blanca,barbas del capitan,0,46.000000,2,-0.045211
7,7,arg,bariloche,pared blanca,jack sparrow,0,51.000000,1,-0.045211
8,8,arg,bariloche,pared blanca,lenguita de gato,0,53.000000,2,-0.073997
9,9,arg,barrosa,matrix,matrix,0,48.500000,0,-0.045211


In [6]:
prub = mountain.sample(1000)

In [5]:
prub["coord"] = prub["name"].apply(sp.geo_cord)

OK
OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
No OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
No OK
OK
OK
OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
OK
OK
OK
OK
No OK
OK
No OK
OK
OK
OK
No OK
OK
OK
OK
No OK
OK
OK
No OK
OK
OK
No OK
OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
OK
OK
No OK
OK
No OK
OK
OK
No OK
OK
No OK
OK
No OK
OK
No OK
OK
OK
OK
OK
No OK
OK
No OK
OK
No O

In [7]:
prub.isnull().sum()

name_id               0
country               0
crag                  0
sector                0
name                  0
tall_recommend_sum    0
grade_mean            0
cluster               0
rating_tot            0
dtype: int64

In [8]:
mountain["coord"] = mountain["name"].apply(sp.geo_cord)
mountain.to_csv("../data/routes.csv")

In [5]:
mountain = pd.read_csv("../data/routes.csv", index_col=0)

In [6]:
mountain.head()

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot,coord
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.250000,3,-0.045211,"(-31.6262496, -64.4437191)"
1,1,and,montserrat,prohibitivo,mehir,-1,49.000000,2,0.116464,"(33.4308245, 46.4636819)"
2,2,and,montserrat,prohibitivo,pas de la discordia,0,49.000000,2,0.178722,NaN
3,3,and,tartareu,bombo suis,tenedor libre,0,44.333333,3,0.158449,"(-26.4537501, -54.6440551)"
4,4,arg,bandurrias,rincon,tendinitis,1,48.500000,0,0.075797,NaN


In [7]:
df_elim = mountain[mountain["coord"].notnull()]
df_elim.head()

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot,coord
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.250000,3,-0.045211,"(-31.6262496, -64.4437191)"
1,1,and,montserrat,prohibitivo,mehir,-1,49.000000,2,0.116464,"(33.4308245, 46.4636819)"
3,3,and,tartareu,bombo suis,tenedor libre,0,44.333333,3,0.158449,"(-26.4537501, -54.6440551)"
5,5,arg,bariloche,pared blanca,barbaroja,0,49.000000,1,-0.164902,"(-26.18040775, -58.174570446864095)"
7,7,arg,bariloche,pared blanca,jack sparrow,0,51.000000,1,-0.045211,"(56.1161043, 10.2233217)"


In [56]:
type(df_elim["coord"].iloc[1])

str

In [71]:
def limp(looc):
    looc1 = float(df_elim["coord"].iloc[1].split(",")[0].replace("(",""))
    looc2 = float(df_elim["coord"].iloc[1].split(",")[1].replace(")",""))

    return looc1, looc2

In [76]:
df_elim["lat"] = df_elim["coord"].apply(lambda x: float(x.split(",")[0].replace("(","")))
df_elim["lon"] = df_elim["coord"].apply(lambda x: float(x.split(",")[1].replace(")","")))
df_elim.head()

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot,coord,lat,lon
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.250000,3,-0.045211,"(-31.6262496, -64.4437191)",-31.626250,-64.443719
1,1,and,montserrat,prohibitivo,mehir,-1,49.000000,2,0.116464,"(33.4308245, 46.4636819)",33.430824,46.463682
3,3,and,tartareu,bombo suis,tenedor libre,0,44.333333,3,0.158449,"(-26.4537501, -54.6440551)",-26.453750,-54.644055
5,5,arg,bariloche,pared blanca,barbaroja,0,49.000000,1,-0.164902,"(-26.18040775, -58.174570446864095)",-26.180408,-58.174570
7,7,arg,bariloche,pared blanca,jack sparrow,0,51.000000,1,-0.045211,"(56.1161043, 10.2233217)",56.116104,10.223322


In [ ]:
df_elim = df_elim.reset_index(drop=True)

In [17]:
df_elim.to_csv("../data/routes2.csv")

In [2]:
df_elim = pd.read_csv("../data/routes2.csv", index_col=0)

In [3]:
df_elim.head()

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot,coord,lat,lon
0,0,and,montserrat,prohibitivo,diagonal de la x,-1,49.250000,3,-0.045211,"(-31.6262496, -64.4437191)",-31.626250,-64.443719
1,1,and,montserrat,prohibitivo,mehir,-1,49.000000,2,0.116464,"(33.4308245, 46.4636819)",33.430824,46.463682
2,3,and,tartareu,bombo suis,tenedor libre,0,44.333333,3,0.158449,"(-26.4537501, -54.6440551)",-26.453750,-54.644055
3,5,arg,bariloche,pared blanca,barbaroja,0,49.000000,1,-0.164902,"(-26.18040775, -58.174570446864095)",-26.180408,-58.174570
4,7,arg,bariloche,pared blanca,jack sparrow,0,51.000000,1,-0.045211,"(56.1161043, 10.2233217)",56.116104,10.223322


In [3]:
df_elim[(df_elim["grade_mean"]>51) & (df_elim["grade_mean"]<55) & (df_elim["country"] == "esp")]

,name_id,country,crag,sector,name,tall_recommend_sum,grade_mean,cluster,rating_tot,coord,lat,lon
10209,10209,esp,ager,camp de,anis del mono,1,52.666667,2,-0.023801,"(41.441405, 2.2464847)",41.441405,2.246485
10220,10220,esp,ager,camp de,free can do,3,54.727273,2,0.030875,"(40.8412325, 31.1812045)",40.841232,31.181204
10230,10230,esp,ager,camp de,low battery,0,53.000000,2,0.377931,"(32.7701094, -79.9374581)",32.770109,-79.937458
10237,10237,esp,ager,camp de,paris mon amour,0,53.000000,2,0.007381,"(48.8866868, 2.3449549)",48.886687,2.344955
10259,10259,esp,aixorta,nevero,barataria,0,54.000000,0,0.229689,"(29.704083500000003, -90.11145959181923)",29.704084,-90.111460
...,...,...,...,...,...,...,...,...,...,...,...,...
25313,25313,esp,villanueva de cauche,yugoslavia,tio de la vara,1,52.000000,0,0.147700,"(39.3729207, -2.297885)",39.372921,-2.297885
25314,25314,esp,villanueva de cauche,yugoslavia,tomahawk,0,53.000000,2,-0.117713,"(45.4710394, -89.7293317)",45.471039,-89.729332
25409,25409,esp,villanueva del rosario,corral,rubinho,0,53.000000,2,-0.045211,"(-3.80156495, -38.60171613049444)",-3.801565,-38.601716
25425,25425,esp,villanueva del rosario,tajo de la madera,legado,0,52.666667,0,0.220947,"(33.259229950000005, -111.6735251668614)",33.259230,-111.673525


In [4]:
 
ubi = []
pais = []
no = []
for i in range(len(df_elim)):
    try:
        geolocator = Nominatim(user_agent="brand")
        location = geolocator.reverse(f'{df_elim["lat"][i]}, {df_elim["lon"][i]}')
        ubi.append(location.address)
    except:
        print(i)
        ubi.append("unknown")
        no.append(i)

for i in ubi:
    pais.append(i.split(", ")[-1])

df_elim["pais"] = pais

df_elim.to_csv("../data/routes2.csv")

209
1769
3678
4696
4899
6117
6202
7002
8896
9614
10265
11906
12788
13289
13343
13394
14563
15505
15769
16271
17747
19081
20351
21317
22207
22333
22945


In [13]:
df_elim["pais"].unique()

array(['Argentina', 'ایران', 'Danmark', 'Deutschland', 'Chile',
       'United States', 'Lietuva', 'Perú', 'Canada', 'Italia', 'France',
       'India', 'Polska', 'Colombia', 'Honduras', 'México', 'España',
       'ישראל', 'Madagasikara / Madagascar', 'Guyana', 'Россия', '中国',
       'Brasil', 'Magyarország', 'België / Belgique / Belgien',
       'Philippines', 'Costa Rica', 'United Kingdom', '日本', 'Україна',
       'Ελλάς', 'Australia', 'Éire / Ireland', 'ประเทศไทย', 'Malaysia',
       'Suomi / Finland', 'ព្រះរាជាណាចក្រ\u200bកម្ពុជា', 'नेपाल',
       'پاکستان', 'New Zealand/Aotearoa',
       'Schweiz/Suisse/Svizzera/Svizra', 'Hrvatska', 'Indonesia',
       'Nederland', 'Österreich',
       'Bosna i Hercegovina / Босна и Херцеговина', 'العراق', 'Portugal',
       'Ghana', 'Србија', 'Saint Lucia', 'Sverige', 'unknown',
       'Burkina Faso', '臺灣', 'السودان',
       'République démocratique du Congo', 'Palestinian Territory',
       'Namibia', 'Kenya', 'Türkiye', 'Guatemala', 'South Afri

In [ ]:
for i in no:
    print(df_elim["country"][i])

In [ ]:
df_elim["country"] = pais

In [8]:
df_elim["country"].value_counts()

esp    6030
ita    3289
usa    2207
fra    1436
deu    1241
aut    1100
grc    1089
pol     969
bra     603
tur     405
svn     397
che     389
aus     359
hrv     357
rom     336
zaf     330
nor     312
can     278
mex     241
prt     229
swe     220
bel     215
tha     214
bgr     199
ukr     172
gbr     163
nzl     160
rus     145
srb      89
hun      70
arg      63
chn      63
ven      55
isr      48
lao      28
svk      22
chl      14
lux      14
cze      11
isl      11
mar      10
dnk      10
per       9
fin       4
mlt       4
vnm       4
mkd       4
and       3
jor       3
ind       2
msr       2
twn       2
reu       2
col       1
bih       1
phl       1
nld       1
Name: country, dtype: int64

In [3]:
scaler = pd.read_csv("../data/climber_df.csv")

In [10]:
scaler.head(5)

,country,sex,height,weight,age,years_cl,date_first,date_last,grades_count,grades_first,grades_last,grades_max,grades_mean,year_first,year_last
user_id,,,,,,,,,,,,,,,
1,SWE,0,177,73,41.0,21,1999-02-06 23:00:00,2001-07-31 22:00:00,84,36,55,62,46.750000,1999,2001
3,SWE,0,180,78,44.0,22,1999-03-31 22:00:00,2000-07-19 22:00:00,12,53,51,59,52.833333,1999,2000
4,SWE,1,165,58,33.0,16,2004-06-30 22:00:00,2009-05-26 22:00:00,119,53,49,64,53.890756,2004,2009
10,SWE,0,167,63,52.0,25,2000-01-14 23:00:00,2017-06-01 22:00:00,298,53,49,63,49.406040,2000,2017
16,NOR,0,177,68,44.0,21,1998-02-27 23:00:00,2010-05-13 22:00:00,5,53,49,53,51.400000,1998,2010


In [7]:
mountain.isnull().sum()

name_id                   0
country                   0
crag                      0
sector                    0
name                      0
tall_recommend_sum        0
grade_mean                0
cluster                   0
rating_tot                0
coord                 32222
dtype: int64

In [8]:
scaler.isnull().sum()

country         0
sex             0
height          0
weight          0
age             0
years_cl        0
date_first      0
date_last       0
grades_count    0
grades_first    0
grades_last     0
grades_max      0
grades_mean     0
year_first      0
year_last       0
dtype: int64

In [9]:
mountain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55858 entries, 0 to 55857
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name_id             55858 non-null  int64  
 1   country             55858 non-null  object 
 2   crag                55858 non-null  object 
 3   sector              55858 non-null  object 
 4   name                55858 non-null  object 
 5   tall_recommend_sum  55858 non-null  int64  
 6   grade_mean          55858 non-null  float64
 7   cluster             55858 non-null  int64  
 8   rating_tot          55858 non-null  float64
 9   coord               23636 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 4.7+ MB


In [10]:
scaler.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10927 entries, 1 to 67020
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       10927 non-null  object 
 1   sex           10927 non-null  int64  
 2   height        10927 non-null  int64  
 3   weight        10927 non-null  int64  
 4   age           10927 non-null  float64
 5   years_cl      10927 non-null  int64  
 6   date_first    10927 non-null  object 
 7   date_last     10927 non-null  object 
 8   grades_count  10927 non-null  int64  
 9   grades_first  10927 non-null  int64  
 10  grades_last   10927 non-null  int64  
 11  grades_max    10927 non-null  int64  
 12  grades_mean   10927 non-null  float64
 13  year_first    10927 non-null  int64  
 14  year_last     10927 non-null  int64  
dtypes: float64(2), int64(10), object(3)
memory usage: 1.3+ MB


In [11]:
scaler.shape

(10927, 15)

In [12]:
mountain.describe(include = "object").T

,count,unique,top,freq
country,55858,59,esp,15293
crag,55858,2245,frankenjura,1596
sector,55858,2463,vermell,316
name,55858,48468,sin nombre,31
coord,23636,18287,"(-44.03352395, -73.81877445387619)",31


In [13]:
scaler.describe(include = "object").T

,count,unique,top,freq
country,10927,26,ESP,1306
date_first,10927,4443,2005-12-31 23:00:00,32
date_last,10927,3882,2017-09-09 22:00:00,118


In [14]:
mountain.describe().T

,count,mean,std,min,25%,50%,75%,max
name_id,55858.0,2.792850e+04,16124.960005,0.000000,13964.250000,27928.500000,41892.750000,55857.000000
tall_recommend_sum,55858.0,2.304057e-02,0.965519,-16.000000,0.000000,0.000000,0.000000,24.000000
grade_mean,55858.0,4.679925e+01,9.168034,29.000000,39.937500,46.000000,53.000000,77.333333
cluster,55858.0,2.210588e+00,1.708326,0.000000,2.000000,2.000000,2.000000,8.000000
rating_tot,55858.0,1.653667e-17,0.140403,-0.182202,-0.073475,-0.042848,0.045295,1.044532


In [15]:
scaler.describe().T

,count,mean,std,min,25%,50%,75%,max
sex,10927.0,0.125103,0.330851,0.0,0.0,0.000000,0.000000,1.000000
height,10927.0,176.152009,8.508669,137.0,171.0,177.000000,182.000000,202.000000
weight,10927.0,67.608676,9.677316,40.0,63.0,68.000000,73.000000,93.000000
age,10927.0,33.333852,7.590989,12.0,28.0,33.000000,38.000000,69.000000
years_cl,10927.0,12.672188,6.108451,1.0,8.0,12.000000,17.000000,29.000000
grades_count,10927.0,79.794546,141.411297,1.0,8.0,28.000000,90.000000,2445.000000
grades_first,10927.0,45.648851,9.478173,28.0,38.0,46.000000,53.000000,75.000000
grades_last,10927.0,46.983802,9.418087,28.0,40.0,48.000000,53.000000,77.000000
grades_max,10927.0,53.764437,9.679533,29.0,46.0,55.000000,62.000000,77.000000
grades_mean,10927.0,45.505055,7.891356,28.5,39.4,45.151899,51.210084,75.272727


In [4]:
df_elim = scaler[(scaler["year_first"] == 0)]
df_elim.shape

(2, 16)

In [5]:
scaler.drop(df_elim.index, axis = 0, inplace = True)

In [6]:
scaler["year_first"].min()

1095

In [7]:
df_elim2 = scaler[(scaler["year_first"] < 1900) & (scaler["year_last"] > scaler["year_first"]+ 80)]
df_elim2.shape

(1, 16)

In [8]:
scaler.drop(df_elim2.index, axis = 0, inplace = True)

In [9]:
scaler.to_csv("../data/climber_df.csv")